# "Observatorio de sueldos en Chile"
> Observatorio de sueldos en Chile

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

Los datos de sueldos en Chile se pueden obtener de la [encuesta suplementaria de ingresos (ESI)]
(https://www.ine.gob.cl/estadisticas/sociales/ingresos-y-gastos/encuesta-suplementaria-de-ingresos). La última ESI disponible es la del año 2021 y se puede descargar [aquí](https://www.ine.gob.cl/docs/default-source/encuesta-suplementaria-de-ingresos/bbdd/csv_esi/2021/esi-2021---personas.csv?sfvrsn=d03ae552_4&download=true).

Una vez que hemos descargado los datos podemos acceder a ellos:

In [1]:
#collapse-hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, display_html, HTML

# download the data
data_raw = pd.read_csv("esi-2021---personas.csv", encoding="ISO-8859-1", low_memory=False, index_col=0)
data_raw.head(3)

,idrph,nro_linea,edad,tramo_edad,sexo,parentesco,curso,nivel,termino_nivel,estudia_actual,...,mes_central,ano_encuesta,mes_encuesta,region,r_p_c,estrato,tipo,conglomerado,id_identificacion,hogar
1,14565,4,36,5.0,1,4,2,4,2,2,...,11,2021,10,8,8205,8200111,1,820510025,102542.0,1
2,24946,2,57,9.0,2,3,2,10,1,2,...,11,2021,12,13,13123,13291,1,13123000144956,69274.0,1
3,41897,2,64,10.0,2,2,3,8,1,2,...,11,2021,10,16,16302,16300120,3,1630220015,80521.0,1


La encuesta considera sólo las personas ocupadas con más de 1 mes en el empleo actual.
Podemos acceder al [Manual y Guía de Variables ESI](https://www.ine.gob.cl/docs/default-source/encuesta-suplementaria-de-ingresos/bbdd/manual-y-gu%C3%ADa-de-variables/2020/personas-esi-2020.pdf?sfvrsn=f196cb4e_4) y ver que la variable que nos interesa es `ocup_ref` donde el valor 1 corresponde a los "Ocupados con más
de 1 mes en el empleo actual".

In [2]:
data = data_raw[data_raw["ocup_ref"] == 1].copy()

Del [Manual y Guía de Variables ESI](https://www.ine.gob.cl/docs/default-source/encuesta-suplementaria-de-ingresos/bbdd/manual-y-gu%C3%ADa-de-variables/2020/personas-esi-2020.pdf?sfvrsn=f196cb4e_4), vemos que debemos considerar el factor de expansión que corresponde a la variable `fact_cal_esi`. De esta forma encontramos que el número de personas ocupadas es 8.243.580.

In [3]:
n_ocupados = data['fact_cal_esi'].sum()
print(f"Número de personas ocupadas: {n_ocupados:,.0f}".replace(',','.'))

Número de personas ocupadas: 8.243.580


Esto coincide con el valor entregado por INE por lo que vamos por buen camino :-)


Para calcular el sueldo promedio, según el Manual y Guía de Variables ESI, debemos utilizar la variable `ing_t_p` que corresponde a "Ingresos del trabajo principal" y debemos utilizar nuevamente el factor de expansión.

In [4]:
def ingreso_promedio(data):
    n_ocupados = data['fact_cal_esi'].sum()
    promedio = (data['ing_t_p']*data['fact_cal_esi']).sum()/n_ocupados
    return int(np.round(promedio))

In [5]:
print(f"Ingreso promedio mensual: ${ingreso_promedio(data):,.0f}".replace(',','.'))

Ingreso promedio mensual: $681.039


In [6]:
ocupados_hombres = data[data['sexo'] == 1]
ocupadas_mujeres = data[data['sexo'] == 2]
print(f"Porcentaje de hombres: {100*ocupados_hombres['fact_cal_esi'].sum()/n_ocupados:.1f}%")
print(f"Porcentaje de mujeres: {100*ocupadas_mujeres['fact_cal_esi'].sum()/n_ocupados:.1f}%")

Porcentaje de hombres: 58.2%
Porcentaje de mujeres: 41.8%


In [7]:
print(f"Ingreso promedio mensual para hombres: ${ingreso_promedio(ocupados_hombres):,.0f}".replace(',','.'))
print(f"Ingreso promedio mensual para mujeres: ${ingreso_promedio(ocupadas_mujeres):,.0f}".replace(',','.'))

Ingreso promedio mensual para hombres: $749.046
Ingreso promedio mensual para mujeres: $586.178


In [8]:
data['sexo'] = data['sexo'].map({1: 'hombre', 2: 'mujer'})
data.groupby('sexo').apply(lambda x: int(np.round(ingreso_promedio(x))))

sexo
hombre    749046
mujer     586178
dtype: int64

In [10]:
map_regiones = {
    1: "Tarapacá",
    2: "Antofagasta",
    3: "Atacama",
    4: "Coquimbo",
    5: "Valparaíso",
    6: "O'Higgins",
    7: "Maule",
    8: "Biobío",
    9: "La Araucanía",
    10: "Los Lagos",
    11: "Aysén",
    12: "Magallanes",
    13: "Metropolitana",
    14: "Los Ríos",
    15: "Arica y Parinacota",
    16: "Ñuble",
    99: "Región no identificada"
}

In [12]:
data["region"] = data["region"].map(map_regiones)

In [13]:
data.groupby('region').apply(lambda x: int(np.round(ingreso_promedio(x))))

region
Antofagasta           765318
Arica y Parinacota    582646
Atacama               649946
Aysén                 748998
Biobío                574946
Coquimbo              603089
La Araucanía          533858
Los Lagos             552445
Los Ríos              576430
Magallanes            844329
Maule                 534284
Metropolitana         780454
O'Higgins             567721
Tarapacá              672109
Valparaíso            601402
Ñuble                 543780
dtype: int64

Espero que este post ayude a clarificar dudas respecto al porqué utilizar la tasa de mortalidad ajustada y el cálculo de ésta. Otras dudas o sugerencias pueden hacerlas enviándome un mensaje en Twitter: [alonsosilva](https://twitter.com/alonsosilva)